In [15]:
#import dependencies

import pandas as pd
import numpy as np
import itertools
import os
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn import datasets
from sklearn.preprocessing import LabelBinarizer, LabelEncoder

In [19]:
# import data
df_requests = pd.read_excel('frc_data.xlsx', sheet_name='frc data')


In [20]:
df_requests.head()

,ID,Submission Date,Resolution Date,FRC Owner,Request,Requestor,Request Description,Status,Urgent?,Reopen?,...,Incident / PME / Project # (Where Appl.),Created,Created By,Modified,Last Modified By,Submission Month,res date,Comments/Notes,Item Type,Path
0,14632,2018-10-18,2018-10-18,"Iannacone, Kristine",Account locked in TAP,"Melaniphy,Kevin M","\n​\nTeam,\n \nMy policy is locked in TAP, can...",Closed,No,No,...,NaN,2018-10-18 16:46:51,"Iannacone,Kristine",2018-10-18 17:02:04,"Iannacone,Kristine",201810,10/18/2018 17:02,"\nGood afternoon Kevin,\nThe TAP helpdesk will...",Item,ccm/FRC/Lists/TEST
1,14631,2018-10-18,NaT,"Osswald, Ernst",Can we add Tow & Labor to auto without PPTs?,"Loquiao,Jeffrey E","\n​\nHello,\n \nCan you please confirm if we a...",Open,No,No,...,NaN,2018-10-18 16:42:54,"Iannacone,Kristine",2018-10-18 16:42:54,"Iannacone,Kristine",201810,-,NaN,Item,ccm/FRC/Lists/TEST
2,14630,2018-10-18,NaT,"Ewing, Jamie",BPP rates higher than Property rates,"Williams,Gina","\n​\nHello,\n \nCan you look into why the BPP ...",Open,No,No,...,NaN,2018-10-18 16:32:44,"Iannacone,Kristine",2018-10-18 16:32:44,"Iannacone,Kristine",201810,-,NaN,Item,ccm/FRC/Lists/TEST
3,14629,2018-10-18,NaT,"Ewing, Jamie",Problem saving files to EPC/DMF,"Albright,Kevin T.",\n​I thought I read something recently on this...,Open,No,No,...,NaN,2018-10-18 16:25:37,"Iannacone,Kristine",2018-10-18 16:25:37,"Iannacone,Kristine",201810,-,NaN,Item,ccm/FRC/Lists/TEST
4,14628,2018-10-18,NaT,"Iannacone, Kristine",Need point of contact for WC claims issues,"Sterling,Heather","\n​\nHello,\n \nI have a insured who would lik...",Open,No,No,...,NaN,2018-10-18 16:16:11,"Iannacone,Kristine",2018-10-18 16:16:11,"Iannacone,Kristine",201810,-,NaN,Item,ccm/FRC/Lists/TEST


In [21]:
monoline_list = ['Equipment Breakdown', 'Inland Marine', 'International', 'Ocean Marine']
package_general_liability_list = ['Connect CNP', 'General Liability', 'Multiline']
package_property_list = ['Paramount Package', 'Property']
condensed_lob_list = []
for row in df_requests['LOB']:
    if row in monoline_list:
        condensed_lob_list.append('Monoline')
    elif row in package_general_liability_list:
        condensed_lob_list.append('Package General Liability')
    elif row in package_property_list:
        condensed_lob_list.append('Package Property')
    else:
        condensed_lob_list.append(row)
        
df_requests['Updated LOB'] = condensed_lob_list

In [22]:
df_requests.head()

,ID,Submission Date,Resolution Date,FRC Owner,Request,Requestor,Request Description,Status,Urgent?,Reopen?,...,Created,Created By,Modified,Last Modified By,Submission Month,res date,Comments/Notes,Item Type,Path,Updated LOB
0,14632,2018-10-18,2018-10-18,"Iannacone, Kristine",Account locked in TAP,"Melaniphy,Kevin M","\n​\nTeam,\n \nMy policy is locked in TAP, can...",Closed,No,No,...,2018-10-18 16:46:51,"Iannacone,Kristine",2018-10-18 17:02:04,"Iannacone,Kristine",201810,10/18/2018 17:02,"\nGood afternoon Kevin,\nThe TAP helpdesk will...",Item,ccm/FRC/Lists/TEST,Not LOB Specific
1,14631,2018-10-18,NaT,"Osswald, Ernst",Can we add Tow & Labor to auto without PPTs?,"Loquiao,Jeffrey E","\n​\nHello,\n \nCan you please confirm if we a...",Open,No,No,...,2018-10-18 16:42:54,"Iannacone,Kristine",2018-10-18 16:42:54,"Iannacone,Kristine",201810,-,NaN,Item,ccm/FRC/Lists/TEST,Auto
2,14630,2018-10-18,NaT,"Ewing, Jamie",BPP rates higher than Property rates,"Williams,Gina","\n​\nHello,\n \nCan you look into why the BPP ...",Open,No,No,...,2018-10-18 16:32:44,"Iannacone,Kristine",2018-10-18 16:32:44,"Iannacone,Kristine",201810,-,NaN,Item,ccm/FRC/Lists/TEST,Package Property
3,14629,2018-10-18,NaT,"Ewing, Jamie",Problem saving files to EPC/DMF,"Albright,Kevin T.",\n​I thought I read something recently on this...,Open,No,No,...,2018-10-18 16:25:37,"Iannacone,Kristine",2018-10-18 16:25:37,"Iannacone,Kristine",201810,-,NaN,Item,ccm/FRC/Lists/TEST,Package Property
4,14628,2018-10-18,NaT,"Iannacone, Kristine",Need point of contact for WC claims issues,"Sterling,Heather","\n​\nHello,\n \nI have a insured who would lik...",Open,No,No,...,2018-10-18 16:16:11,"Iannacone,Kristine",2018-10-18 16:16:11,"Iannacone,Kristine",201810,-,NaN,Item,ccm/FRC/Lists/TEST,Workers Comp


In [23]:
target = df_requests["Updated LOB"]
target_names = ["Auto", "Workers Comp", "Package Property", "Package General Liability", "Not LOB Specific", "Monoline", "Umbrella"]

In [25]:
data = df_requests.drop("Updated LOB", axis=1)
feature_names = df_requests["Request Description"]
data.head()

,ID,Submission Date,Resolution Date,FRC Owner,Request,Requestor,Request Description,Status,Urgent?,Reopen?,...,Incident / PME / Project # (Where Appl.),Created,Created By,Modified,Last Modified By,Submission Month,res date,Comments/Notes,Item Type,Path
0,14632,2018-10-18,2018-10-18,"Iannacone, Kristine",Account locked in TAP,"Melaniphy,Kevin M","\n​\nTeam,\n \nMy policy is locked in TAP, can...",Closed,No,No,...,NaN,2018-10-18 16:46:51,"Iannacone,Kristine",2018-10-18 17:02:04,"Iannacone,Kristine",201810,10/18/2018 17:02,"\nGood afternoon Kevin,\nThe TAP helpdesk will...",Item,ccm/FRC/Lists/TEST
1,14631,2018-10-18,NaT,"Osswald, Ernst",Can we add Tow & Labor to auto without PPTs?,"Loquiao,Jeffrey E","\n​\nHello,\n \nCan you please confirm if we a...",Open,No,No,...,NaN,2018-10-18 16:42:54,"Iannacone,Kristine",2018-10-18 16:42:54,"Iannacone,Kristine",201810,-,NaN,Item,ccm/FRC/Lists/TEST
2,14630,2018-10-18,NaT,"Ewing, Jamie",BPP rates higher than Property rates,"Williams,Gina","\n​\nHello,\n \nCan you look into why the BPP ...",Open,No,No,...,NaN,2018-10-18 16:32:44,"Iannacone,Kristine",2018-10-18 16:32:44,"Iannacone,Kristine",201810,-,NaN,Item,ccm/FRC/Lists/TEST
3,14629,2018-10-18,NaT,"Ewing, Jamie",Problem saving files to EPC/DMF,"Albright,Kevin T.",\n​I thought I read something recently on this...,Open,No,No,...,NaN,2018-10-18 16:25:37,"Iannacone,Kristine",2018-10-18 16:25:37,"Iannacone,Kristine",201810,-,NaN,Item,ccm/FRC/Lists/TEST
4,14628,2018-10-18,NaT,"Iannacone, Kristine",Need point of contact for WC claims issues,"Sterling,Heather","\n​\nHello,\n \nI have a insured who would lik...",Open,No,No,...,NaN,2018-10-18 16:16:11,"Iannacone,Kristine",2018-10-18 16:16:11,"Iannacone,Kristine",201810,-,NaN,Item,ccm/FRC/Lists/TEST


In [26]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=42)

In [27]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
clf.score(X_test, y_test)

ValueError: could not convert string to float: 'ccm/FRC/Lists/TEST'

In [28]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


ValueError: could not convert string to float: 'ccm/FRC/Lists/TEST'